# Library

In [1]:
import os
from dotenv import load_dotenv 
import pandas_datareader.data as web
from langchain import hub
from langchain.agents import Tool, create_react_agent 
from langchain.tools import DuckDuckGoSearchResults
from langchain_openai import OpenAI
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.schema import AgentFinish

# 環境変数の読み込み
load_dotenv()  # .envファイルから環境変数を読み込み
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# GPTモデルの設定
model_name = 'gpt-4o-mini-2024-07-18'  # 'gpt-4o-2024-08-06'

# OpenAI APIのキーをセット
llm = OpenAI(temperature=0.5, api_key=os.getenv('OPENAI_API_KEY'))

# Function

In [2]:
# DuckDuckGo検索ツールの作成
search_tool = Tool(
    name="DuckDuckGo Search",
    func=DuckDuckGoSearchResults(),
    description="Use this tool to search for general information on the web."
)

# 独自Python REPLツールの作成
def execute_python_code(code: str):
    try:
        # 入力されたコードを実行
        exec_globals = {}
        exec(code, exec_globals)
        return exec_globals.get("result", "No result variable found.")
    except Exception as e:
        return f"Error: {str(e)}"

python_tool = Tool(
    name="Python Executor",
    func=execute_python_code,
    description="Execute Python code."
)

# 株価取得ツールの作成
def get_stock_price(code: str, date_from: str, date_to: str):
    # 日本用のコードに変換
    code = code + '.JP'
    # stooqからデータ取得
    df = web.DataReader(code, data_source='stooq', start=date_from, end=date_to)
    
    if not df.empty:
        # 終値を取得し、JSON形式で返す
        return df['Close'].to_json(orient='table')
    else:
        return f"Could not retrieve stock price for {code} from {date_from} to {date_to}."

def get_stock_price_range(arguments):
    res = get_stock_price(
        code=arguments.get('code'),
        date_from=arguments.get('date_from'),
        date_to=arguments.get('date_to')
    )
    return res

stock_tool = Tool(
    name="Stock Price Checker",
    func=get_stock_price,
    description="""Retrieve stock prices for a specific company (ticker symbol) for a given time period or at a specific point in time. 
Only use this tool for queries explicitly related to stock prices or financial data."""
)

In [9]:
search_result = search_tool.run("highest mountains in the world and highest mountains in Japan")
print(search_result)

snippet: Almost all mountains in the list are located in the Himalaya and Karakoram ranges to the south and west of the Tibetan plateau. All peaks 7,000 m (23,000 ft) or higher are located in East, Central or South Asia in a rectangle edged by Noshaq (7,492 m or 24,580 ft) on the Afghanistan-Pakistan border in the west, Jengish Chokusu (Tuōmù'ěr Fēng, 7,439 m or 24,406 ft) on the Kyrgyzstan ..., title: List of highest mountains on Earth - Wikipedia, link: https://en.wikipedia.org/wiki/List_of_highest_mountains_on_Earth, snippet: Mount Fuji, highest mountain in Japan. It rises to 12,388 feet (3,776 meters) near the Pacific coast of central Honshu, about 60 miles (100 km) west of the Tokyo-Yokohama metropolitan area. It is a volcano that has been dormant since its last eruption (1707) but is still generally classified as active by geologists., title: Mount Fuji | Facts, Height, Location, & Eruptions | Britannica, link: https://www.britannica.com/place/Mount-Fuji, snippet: Standing as the

# Custom Output Parser

In [18]:
# カスタム出力パーサーの作成
class CustomOutputParser(ReActSingleInputOutputParser):
    def parse(self, text: str):
        # "Final Answer:" があれば、その部分を最終結果とみなす
        if "Final Answer:" in text:
            final_answer = text.split("Final Answer:")[-1].strip()
            return AgentFinish({"output": final_answer}, text)
        
        # "Action"が含まれている場合はアクションを実行する
        if "Action:" in text:
            action_text = text.split("Action:")[-1].strip()
            # 例として "DuckDuckGo Search" を処理
            if "DuckDuckGo Search" in action_text:
                action_input = action_text.split("Action Input:")[-1].strip()
                return AgentAction("search_tool", action_input, text)
            # 他のアクションも追加できる
            elif "Python" in action_text:
                action_input = action_text.split("Action Input:")[-1].strip()
                return AgentAction("python_tool", action_input, text)
        
        # "Final Answer"も"Action"もない場合はそのまま出力
        return AgentFinish({"output": text.strip()}, text)

# Prompt Template

In [4]:
prompt = hub.pull("hwchase17/react")
print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


# ReAct base Agent

In [10]:
# ReActベースのエージェントを初期化
tools = [search_tool, python_tool, stock_tool]
tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])
tool_names = ", ".join([tool.name for tool in tools])

# カスタム出力パーサーを使用してエージェントを作成
output_parser = CustomOutputParser()
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    output_parser=output_parser
)

# Question

In [19]:
question = "世界で一番高い山は日本で一番高い山の何倍の高さですか？"

# `intermediate_steps` を初期化してエージェントを呼び出す
response = agent.invoke({
    "input": question,
    "agent_scratchpad": "",
    "intermediate_steps": []
})

print("Agent response received")
print(response)

# AgentFinish オブジェクトの return_values から結果を取得
formatted_output = f"""
Result:
-----------------------------------------
{response.return_values['output']}
"""

print(formatted_output)

Agent response received
return_values={'output': 'I should use DuckDuckGo Search to find information on the tallest mountains in the world and in Japan.\nAction: DuckDuckGo Search\nAction Input: "tallest mountains in the world" and "tallest mountain in Japan"'} log=' I should use DuckDuckGo Search to find information on the tallest mountains in the world and in Japan.\nAction: DuckDuckGo Search\nAction Input: "tallest mountains in the world" and "tallest mountain in Japan"'

Result:
-----------------------------------------
I should use DuckDuckGo Search to find information on the tallest mountains in the world and in Japan.
Action: DuckDuckGo Search
Action Input: "tallest mountains in the world" and "tallest mountain in Japan"



In [7]:
question = "Sumitomo Mitsui Financial Groupはどのような企業ですか？2024年8月30日時点の株価は？"

# `intermediate_steps` を初期化してエージェントを呼び出す
response = agent.invoke({
    "input": question,
    "agent_scratchpad": "",
    "intermediate_steps": []
})

# AgentFinish オブジェクトの return_values から結果を取得
formatted_output = f"""
Result:
-----------------------------------------
{response.return_values['output']}
"""

print(formatted_output)


Result:
-----------------------------------------
We should search for information about Sumitomo Mitsui Financial Group and then use the Stock Price Checker to retrieve the stock price on August 30th, 2024.
Action: DuckDuckGo Search
Action Input: "Sumitomo Mitsui Financial Group"

